In [1]:
import pandas as pd
import databaseconfig as dbconnect
Branch_General_Profile = pd.read_csv("resources/Branch_General_Profile.csv")

In [2]:
Branch_General_Profile.head()

,ID,Branch Name,Tier,Address,Telephone,Postal Code,FSA,NBHD Number,NBHD Name,Ward Number,Ward Region
0,AG,Agincourt,DL,155 Bonis Ave,(416) 396-8943,M1T 3W6,M1T,118,Tam O'Shanter-Sullivan,40,Scarborough-Agincourt
1,ACD,Albert Campbell,DL,496 Birchmount Rd,(416) 396-8890,M1K 1N8,M1K,120,Clairlea-Birchmount,35,Scarborough-Southwest
2,AB,Albion,DL,1515 Albion Rd,(416) 394-5170,M9V 1B2,M9V,2,Mount Olive-Silverstone-Jamestown,1,Etobicoke North
3,AD,Alderwood,NL,2 Orianna Dr,(416) 394-5310,M8W 4Y1,M8W,20,Alderwood,6,Etobicoke Lakeshore
4,AP,Amesbury Park,NL,1565 Lawrence Ave W,(416) 395-5420,M6L 1A8,M6L,30,Brookhaven-Amesbury,12,York South-Weston


In [3]:
Branch_General_Profile.keys()

Index(['ID', 'Branch Name', 'Tier', 'Address', 'Telephone ', 'Postal Code',
       'FSA', 'NBHD Number', 'NBHD Name', 'Ward Number', 'Ward Region'],
      dtype='object')

In [4]:
nbhd_df = Branch_General_Profile[['NBHD Number','NBHD Name']]
nbhd_df = nbhd_df.sort_values('NBHD Number')
print(len(nbhd_df))
nbhd_df.drop_duplicates(subset = 'NBHD Number', keep = 'first', inplace=True)
print(len(nbhd_df))
nbhd_df = nbhd_df.rename(columns={'NBHD Number':'id','NBHD Name':'name'})
nbhd_df = nbhd_df.reset_index(drop=True)
nbhd_df.head()

105
85


,id,name
0,1,West Humber-Clairville
1,2,Mount Olive-Silverstone-Jamestown
2,4,Rexdale-Kipling
3,7,Willowridge-Martingrove-Richview
4,11,Eringate-Centennial-West Deane


In [5]:
ward_region_df = Branch_General_Profile[['Ward Number','Ward Region']]
ward_region_df = ward_region_df.sort_values('Ward Number')
print(len(ward_region_df))
ward_region_df.drop_duplicates(subset = 'Ward Number', keep = 'first', inplace=True)
print(len(ward_region_df))
ward_region_df = ward_region_df.rename(columns={'Ward Number':'id','Ward Region':'name'})
ward_region_df = ward_region_df.reset_index(drop=True)
ward_region_df.head()

105
44


,id,name
0,1,Etobicoke North
1,2,Etobicoke North
2,3,Etobicoke Centre
3,4,Etobicoke Centre
4,5,Etobicoke- Lakeshore


In [6]:
Branch_General_Profile_df = Branch_General_Profile[['ID','Branch Name','Tier','Address','Telephone ','Postal Code','FSA','NBHD Number','Ward Number']]
Branch_General_Profile_df = Branch_General_Profile_df.rename(columns={'ID':'id','Branch Name':'name','Tier':'tier','Address':'address','Telephone ':'contact_no','Postal Code':'postal_code','FSA':'fsa','NBHD Number':'nbhd_no','Ward Number':'ward_no'})
Branch_General_Profile_df.head()

,id,name,tier,address,contact_no,postal_code,fsa,nbhd_no,ward_no
0,AG,Agincourt,DL,155 Bonis Ave,(416) 396-8943,M1T 3W6,M1T,118,40
1,ACD,Albert Campbell,DL,496 Birchmount Rd,(416) 396-8890,M1K 1N8,M1K,120,35
2,AB,Albion,DL,1515 Albion Rd,(416) 394-5170,M9V 1B2,M9V,2,1
3,AD,Alderwood,NL,2 Orianna Dr,(416) 394-5310,M8W 4Y1,M8W,20,6
4,AP,Amesbury Park,NL,1565 Lawrence Ave W,(416) 395-5420,M6L 1A8,M6L,30,12


In [7]:
from sqlalchemy import create_engine
rds_connection_string = 'postgres:'+dbconnect.postgres['passwd']+'@localhost:5432/toronto_library_db'
engine = create_engine(f'postgresql://{rds_connection_string}')
tables = list(engine.table_names())
tables.sort()
tables

['active_cardholders',
 'age_group',
 'annual_visits',
 'branch',
 'catchment_population',
 'circulation',
 'collection_size',
 'events',
 'hours_of_operation',
 'nbhd',
 'new_registrations',
 'registered_cardholders',
 'ward_region',
 'workstation_users',
 'workstations']

In [10]:
tables[12]

'ward_region'

In [12]:
ward_region_df.to_sql(name=tables[12],con=engine,if_exists='append',index=False)

In [13]:
pd.read_sql_query('select * from '+tables[12], con=engine).head()

,id,name
0,1,Etobicoke North
1,2,Etobicoke North
2,3,Etobicoke Centre
3,4,Etobicoke Centre
4,5,Etobicoke- Lakeshore


In [16]:
tables[9]

'nbhd'

In [17]:
nbhd_df.to_sql(name=tables[9],con=engine,if_exists='append',index=False)
pd.read_sql_query('select * from '+tables[9], con=engine).head()

,id,name
0,1,West Humber-Clairville
1,2,Mount Olive-Silverstone-Jamestown
2,4,Rexdale-Kipling
3,7,Willowridge-Martingrove-Richview
4,11,Eringate-Centennial-West Deane


In [18]:
Branch_General_Profile_df.to_sql(name=tables[3],con=engine,if_exists='append',index=False)
pd.read_sql_query('select * from '+tables[3], con=engine).head()

,id,name,tier,address,contact_no,postal_code,fsa,nbhd_no,ward_no
0,AG,Agincourt,DL,155 Bonis Ave,(416) 396-8943,M1T 3W6,M1T,118,40
1,ACD,Albert Campbell,DL,496 Birchmount Rd,(416) 396-8890,M1K 1N8,M1K,120,35
2,AB,Albion,DL,1515 Albion Rd,(416) 394-5170,M9V 1B2,M9V,2,1
3,AD,Alderwood,NL,2 Orianna Dr,(416) 394-5310,M8W 4Y1,M8W,20,6
4,AP,Amesbury Park,NL,1565 Lawrence Ave W,(416) 395-5420,M6L 1A8,M6L,30,12
